In [1]:
#Adım 1 – Repo’yu klonla
!git clone https://github.com/guzelTemli/YapayZekaChatbot.git
%cd YapayZekaChatbot


In [ ]:
#Adım 2 – Base + LoRA modelini yükle
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL_ID = "google/gemma-2-2b-it"
ADAPTER_DIR = "models/gemma2b_bilgi_guvenligi_lora_master"  # repo içindeki yol

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()


In [ ]:
#Adım 3 – Soru sor
def ask(question: str, max_new_tokens: int = 200) -> str:
    prompt = f"Soru: {question}\nCevap:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Cevap:" in full_text:
        answer = full_text.split("Cevap:", 1)[-1].strip()
    else:
        answer = full_text.strip()
    return answer

print(ask("BYOD politikasına göre kişisel cihazlarla kurumsal verilere erişirken hangi güvenlik önlemleri alınmalıdır?"))
